# Hadooop
<font size="3">Hadoop was one of the first popular distributed MapReduce programming systems, and really paved the way for Big Data analysis at massive scales on commodity hardware. As described in lectures this week, its primary strengths are three-fold: 1) scalability, 2) programmability, and 3) fault tolerance. Hadoop allows programmers to write simple Java applications that run across thousands of networked machines and which, thanks to its MapReduce abstractions, can recover from complete hardware failure of many of those machines.</font>

# [Spark](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
<font size="3">A successor to Hadoop that emphasizes in-memory caching of data and a more flexible API. While Hadoop regularly persisted replicated data to disk to ensure fault tolerance, Spark instead caches information in memory and ensures that it remembers the transformations that were applied to generate a particular piece of data. Spark also offers more than simple map and reduce transformations.</font>  [RDD](https://spark.apache.org/docs/1.2.1/api/java/org/apache/spark/api/java/JavaPairRDD.html)

Spark Resilient Distributed Dataset (There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.)

<div data-lang="java" class="tab-pane active" id="tab_java_8">

    <p>While most Spark operations work on RDDs containing any type of objects, a few special operations are
only available on RDDs of key-value pairs.
The most common ones are distributed “shuffle” operations, such as grouping or aggregating the elements
by a key.</p>

    <p>In Java, key-value pairs are represented using the
<a href="http://www.scala-lang.org/api/2.11.8/index.html#scala.Tuple2">scala.Tuple2</a> class
from the Scala standard library. You can simply call <code>new Tuple2(a, b)</code> to create a tuple, and access
its fields later with <code>tuple._1()</code> and <code>tuple._2()</code>.</p>

    <p>RDDs of key-value pairs are represented by the
<a href="api/java/index.html?org/apache/spark/api/java/JavaPairRDD.html">JavaPairRDD</a> class. You can construct
JavaPairRDDs from JavaRDDs using special versions of the <code>map</code> operations, like
<code>mapToPair</code> and <code>flatMapToPair</code>. The JavaPairRDD will have both standard RDD functions and special
key-value ones.</p>

    <p>For example, the following code uses the <code>reduceByKey</code> operation on key-value pairs to count how
many times each line of text occurs in a file:</p>

    <figure class="highlight"><pre><code class="language-scala" data-lang="scala"><span></span><span class="nc">JavaRDD</span><span class="o">&lt;</span><span class="nc">String</span><span class="o">&gt;</span> <span class="n">lines</span> <span class="k">=</span> <span class="n">sc</span><span class="o">.</span><span class="n">textFile</span><span class="o">(</span><span class="s">"data.txt"</span><span class="o">);</span>
<span class="nc">JavaPairRDD</span><span class="o">&lt;</span><span class="nc">String</span><span class="o">,</span> <span class="nc">Integer</span><span class="o">&gt;</span> <span class="n">pairs</span> <span class="k">=</span> <span class="n">lines</span><span class="o">.</span><span class="n">mapToPair</span><span class="o">(</span><span class="n">s</span> <span class="o">-&gt;</span> <span class="k">new</span> <span class="nc">Tuple2</span><span class="o">(</span><span class="n">s</span><span class="o">,</span> <span class="mi">1</span><span class="o">));</span>
<span class="nc">JavaPairRDD</span><span class="o">&lt;</span><span class="nc">String</span><span class="o">,</span> <span class="nc">Integer</span><span class="o">&gt;</span> <span class="n">counts</span> <span class="k">=</span> <span class="n">pairs</span><span class="o">.</span><span class="n">reduceByKey</span><span class="o">((</span><span class="n">a</span><span class="o">,</span> <span class="n">b</span><span class="o">)</span> <span class="o">-&gt;</span> <span class="n">a</span> <span class="o">+</span> <span class="n">b</span><span class="o">);</span></code></pre></figure>

    <p>We could also use <code>counts.sortByKey()</code>, for example, to sort the pairs alphabetically, and finally
<code>counts.collect()</code> to bring them back to the driver program as an array of objects.</p>

    <p><strong>Note:</strong> when using custom objects as the key in key-value pair operations, you must be sure that a
custom <code>equals()</code> method is accompanied with a matching <code>hashCode()</code> method.  For full details, see
the contract outlined in the <a href="http://docs.oracle.com/javase/7/docs/api/java/lang/Object.html#hashCode()">Object.hashCode()
documentation</a>.</p>

  </div>

## Methods in Example
>join(JavaPairRDD<K,W> other): <br>
>Return an RDD containing all pairs of elements with matching keys in this and other.




<tr>
  <td> <b>groupByKey</b>([<i>numTasks</i>]) <a name="GroupByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable&lt;V&gt;) pairs. <br>
    <b>Note:</b> If you are grouping in order to perform an aggregation (such as a sum or
      average) over each key, using <code>reduceByKey</code> or <code>aggregateByKey</code> will yield much better
      performance.
    <br>
    <b>Note:</b> By default, the level of parallelism in the output depends on the number of partitions of the parent RDD.
      You can pass an optional <code>numTasks</code> argument to set a different number of tasks.
  </td>
</tr>
<br>
<br>
<tr>
  <td> <b>reduceByKey</b>(<i>func</i>, [<i>numTasks</i>]) <a name="ReduceByLink"></a> </td>
  <td> When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function <i>func</i>, which must be of type (V,V) =&gt; V. Like in <code>groupByKey</code>, the number of reduce tasks is configurable through an optional second argument. </td>
</tr>




## Example 
```java
 /* @param sites The connectivity of the website graph, keyed on unique
 *              website IDs.
 * @param ranks The current ranks of each website, keyed on unique website
 *              IDs.
 * @return The new ranks of the websites graph, using the PageRank
 *         algorithm to update site ranks.
 */
public static JavaPairRDD<Integer, Double> sparkPageRank(
            final JavaPairRDD<Integer, Website> sites,
            final JavaPairRDD<Integer, Double> ranks) {

    	
    	JavaPairRDD<Integer, Double> newRank = sites.join(ranks).flatMapToPair(
    			site -> // (37603,(edu.coursera.distributed.Website@92e3,44.66974022768941))
    			{
    				int siteId = site._1();
    				Tuple2<Website, Double> value = site._2();
    				Website edges = site._2()._1();
    				Double currentRank = site._2()._2();
    				Iterator<Integer> iter = edges.edgeIterator();
    				
    				List<Tuple2<Integer, Double>> contribs = new ArrayList<>();
    				while (iter.hasNext())
    				{
    					final int target = iter.next();
    					contribs.add(new Tuple2<>(target, currentRank / (double) edges.getNEdges()));
    				}
    				return contribs;
    			}
    	);
    	return newRank.reduceByKey((Double r1, Double r2) -> r1 + r2)
    			.mapValues(v -> 0.15 + 0.85 * v);
    	

//    	ranks = contribs.reduceByKey(new Sum()).mapValues(sum -> 0.15 + sum * 0.85);
    }
```